$$E_p = \|q_{ball} - q_{target}\| \\E_k = 0; t = 0 \\E_k = \frac{m * V^2}{2} \\ \ddot{q} = \frac{F}{m}\\ F = \nabla U(q, t) = 2 * (q_{ball} - q_{target})\\ \ddot{q} = \frac{2 * (q_{ball} - q_{target})}{m}\\ L(\dot{q}, q, t) = E_k - E_p = \frac{m * V^2}{2} - (q_{ball} - q_{target})^2 \\ \frac{d}{dt} \frac{dL}{d\dot{q}} - \frac{dL}{dq} = 0,\ no\ dissipation\ forces \\ m * \ddot{q} - 2 * (q_{ball} - q_{target}) = 0 \\ F_{rolling\ friction} = \frac{f}{N} * R \\ \ddot{q} = \frac{2 * (q_{ball} - q_{target}) - sign(\dot{q}) * F_{rolling\ friction}}{m}$$

$$ x = x_0 + \dot{x} * dt \\ \dot{x} = \dot{x}_0 + \ddot{x} * dt \\ \ddot{x} = \frac{2 * x - 2 * x_{target} - sign(\dot{x}) * F_{rolling\ friction}}{m} $$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
num_steps = 4000
max_time = 100
time_range = np.linspace(0, max_time, num_steps)

R = 0.05
V = 4 * np.pi * (R ** 3) / 3
ro = 1000
m = V * ro
g = 9.8
f = 0.001
N = m * g
F_fr = f * N / R

x_target = np.array([0, 0], dtype=np.float32)
x0 = np.array([2, 1], dtype=np.float32)
v0 = np.array([0.3, 0.5], dtype=np.float32)
a = lambda x, x_target, m, v: ((2 * (x_target - x) - np.sign(v) * F_fr)  / m)

xs = np.zeros((num_steps, 2))
t0 = 0
x = x0.copy()
v = v0.copy()

for idx, t1 in enumerate(time_range):
    t = t1 - t0
    a_curr = a(x, x_target, m, v)
    v += a_curr * t
    x += v * t
    xs[idx] += x
    t0 = t1
    #print(f'acceleration {a_curr[0]:.2f} {a_curr[1]:.2f}, speed {v[0]:.4f}, coordinate {x[0]:.4f} {x[1]:.4f}, potential {np.mean((x_target - x)**2):.4f}')
    

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(xs[:, 0], xs[:, 1], '.', alpha=0.1)
ax.plot(x0[0], x0[1], 'o')
ax.plot(x_target[0], x_target[1], 'o')
ax.arrow(x0[0], x0[1], v0[0], v0[1])
ax.set_xlim(-x0[0], x0[0] + v0[0] + 0.1);
ax.set_ylim(-x0[1], x0[1] + v0[1] + 0.1);

In [ ]:
np.array([(t / max_time, 1., 1.) for t in time_range]).shape

### Taichi

In [ ]:
import taichi as ti
ti.init(arch=ti.cpu)

screen_res = (800, 400)

@ti.func
def compute_potential(ball_position, target):
    return np.mean((ball_position - target) ** 2)

@ti.func
def compute_acceleration(x, x_target, m, v, F_fr):
    return (2 * (x_target - x) - np.sign(v.to_numpy()) * F_fr)  / m

@ti.func
def update_state():
    a_curr = compute_acceleration(x, x_target_ti, m_ti, v, F_fr_ti)
    v += a_curr * t
    x += v * t

@ti.kernel
def run_simulation():
    for idx, t1 in np.linspace(0, sim_length, num_steps)
    
def render(gui):
    canvas = gui.canvas
    canvas.clear(bg_color)
    pos_np = x.to_numpy()
    gui.circles(pos_np, radius=particle_radius, color=particle_color)
    gui.show()


In [ ]:
x[0][0] - x_target_ti[0][0]

In [ ]:
compute_acceleration(x, x_target_ti, m_ti, v, F_fr_ti)

In [ ]:
num_steps = 4000
max_time = 100
dim = 2

R = 0.05
V = 4 * np.pi * (R ** 3) / 3
ro = 1000
m = V * ro
g = 9.8
f = 0.001
N = m * g
F_fr = f * N / R

x_target = np.array([0, 0], dtype=float)
x0 = np.array([2, 1], dtype=float)
v0 = np.array([0.3, 0.5], dtype=float)

xs = np.zeros((num_steps, dim))
t0 = 0

x = ti.var(dt=ti.f32, shape=(dim))
x_target_ti = ti.var(dt=ti.f32, shape=(dim))
v = ti.var(dt=ti.f32, shape=(dim))
m_ti = ti.var(dt=ti.f32, shape=1)
F_fr_ti = ti.var(dt=ti.f32, shape=1)
t = ti.var(dt=ti.f32, shape=1)

x.from_numpy(x0)
x_target_ti.from_numpy(x_target)
v.from_numpy(v0)
m_ti[0] = m
F_fr_ti[0] = F_fr
gui = ti.GUI('PBF2D', screen_res)

for idx, t1 in enumerate(np.linspace(0, max_time, num_steps)):
    t[0] = t1 - t0
    update_state()
    render(gui)
    xs[idx] += x
    t0 = t1
    #print(f'acceleration {a_curr[0]:.2f} {a_curr[1]:.2f}, speed {v[0]:.4f}, coordinate {x[0]:.4f} {x[1]:.4f}, potential {np.mean((x_target - x)**2):.4f}')


In [ ]:
m